## Introduction:


The Artificial Bee Colony (ABC) algorithm:
*    It's a population-based optimization algorithm inspired by the foraging behavior of honeybees.
*    It consists of three main phases employed bees, onlooker bees, and scout bees.
*   In the employed bee phase, bees exploit the information of food sources.
*   In the onlooker bee phase, bees choose food sources depending on their quality.

*    Finally, in the scout bee phase, bees abandon food sources that have not been visited for a long time and search for new ones.





  

The provided Python code implements the ABC algorithm to solve both minimization and maximization problems. The algorithm iteratively refines a population of candidate solutions, aiming to converge towards the optimal solution.

## 1. Initialization:


In [1]:
import numpy as np

### 1.1 Parameters:


In [2]:
lb = -10  # Lower bounds
ub = 10   # Upper bounds

Np = 5   # Population size
limit = 100  # Limit for scout phase
max_iter = 10  # Maximum iterations

### 1.2 Objective function:


In [3]:
def objective_function(x1, x2, x3, x4):
    return x1**2 + x2**2 + x3**2 + x4**2

### 1.3 Fitness function:


In [4]:
def fitness_function(fx):
    # Fitness function to transform objective values into fitness values
    if fx >= 0:
        return 1 / (1 + fx)
    else:
        return 1 + abs(fx)

### 1.4 Counter function:


In [5]:
def update_counter(new_fx, prev_fx, counter, is_maximization):
    # Counter function to track how many iterations have passed without improvement
    if (is_maximization and new_fx > prev_fx) or (not is_maximization and new_fx < prev_fx):
        return 0
    else:
        return counter + 1

### 1.5 Bound function:


In [6]:
def bound(x_new, lb, ub):
    # Function to ensure the new solution remains within bounds
    return np.clip(x_new, lb, ub)

##2. Employed Bees Phase:


In [7]:
def employed_bees_phase(X, fit, trial, f, lb, ub, is_maximization):
    # Employed bees phase where employed bees explore around their position
    Np, D = X.shape
    for i in range(Np):
        # Select a partner bee
        partner_idx = np.random.choice([idx for idx in range(Np) if idx != i])

        # Choose a dimension to modify
        j = np.random.randint(D)

        # Generate a random number to determine the exploration direction
        phi = np.random.uniform(-1, 1)
        X_new = np.copy(X[i])

        # Update the position
        X_new[j] = X[i][j] + phi * (X[i][j] - X[partner_idx][j])

        # Ensure the new position is within bounds
        X_new = bound(X_new, lb, ub)

        # Calculate the fitness of the new solution
        new_fit = fitness_function(f(*X_new))

        # If the new solution is better, update the position and reset the trial counter
        if new_fit > fit[i]:
            X[i] = X_new
            fit[i] = new_fit
            trial[i] = 0
        else:
            # Otherwise, increment the trial counter
            trial[i] += 1

    return X, fit, trial

##3. Onlooker Bees Phase:


In [8]:
def onlooker_bees_phase(X, fit, trial, f, lb, ub, is_maximization):
    # Onlooker bees phase where bees choose food sources based on their fitness
    Np, D = X.shape
    probabilities = 0.9 * (fit / fit.max()) + 0.1
    m = 0
    while m < Np:
        # Select a bee based on the probabilities of their fitness
        i = np.random.choice(Np, p=probabilities/np.sum(probabilities))
        partner_idx = np.random.choice([idx for idx in range(Np) if idx != i])

        # Choose a dimension to modify
        j = np.random.randint(D)
        phi = np.random.uniform(-1, 1)
        X_new = np.copy(X[i])

        # Update the position
        X_new[j] = X[i][j] + phi * (X[i][j] - X[partner_idx][j])
        X_new = bound(X_new, lb, ub)

        # Calculate the fitness of the new solution
        new_fit = fitness_function(f(*X_new))

        # If the new solution is better, update the position and reset the trial counter
        if new_fit > fit[i]:
            X[i] = X_new
            fit[i] = new_fit
            trial[i] = 0
        else:
            # Otherwise, increment the trial counter
            trial[i] += 1
        m += 1
    return X, fit, trial

##4. Scout Bees Phase:


In [9]:
def scout_bees_phase(X, fit, trial, f, lb, ub, limit):
    # Scout bees phase where abandoned food sources are replaced
    Np, D = X.shape
    for i in range(Np):
        if trial[i] > limit:
            # Replace abandoned food sources with random solutions
            X[i] = np.random.uniform(low=lb, high=ub, size=D)
            fit[i] = fitness_function(f(*X[i]))
            trial[i] = 0
    return X, fit, trial

##5. Memorize the best solution

In [10]:
def memorize_best_solution(X, fit, best_solution, best_fit, is_maximization):
    # Update the best solution found so far
    if is_maximization:
        if np.max(fit) > best_fit:
            best_fit = np.max(fit)
            best_solution = X[np.argmax(fit)]
    else:
        if np.min(fit) < best_fit:
            best_fit = np.min(fit)
            best_solution = X[np.argmin(fit)]
    return best_solution, best_fit

##6. ABC algorithm


In [11]:
def abc_algorithm(f, lb, ub, Np, limit, max_iter, is_maximization=False):
    # Artificial Bee Colony (ABC) algorithm
    D = 4  # Dimension of the problem (x1, x2, x3, x4)

    # Initialize population, fitness, trial counters, and best solution
    X = np.random.uniform(low=lb, high=ub, size=(Np, D))
    fit = np.array([fitness_function(f(*x)) for x in X])
    trial = np.zeros(Np)
    best_solution = X[np.argmax(fit) if is_maximization else np.argmin(fit)]
    best_fit = np.max(fit) if is_maximization else np.min(fit)

    # Main loop
    for iteration in range(max_iter):
        # Employed bees phase
        X, fit, trial = employed_bees_phase(X, fit, trial, f, lb, ub, is_maximization)
        # Onlooker bees phase
        X, fit, trial = onlooker_bees_phase(X, fit, trial, f, lb, ub, is_maximization)
        # Scout bees phase
        X, fit, trial = scout_bees_phase(X, fit, trial, f, lb, ub, limit)
        # Memorize the best solution found so far
        best_solution, best_fit = memorize_best_solution(X, fit, best_solution, best_fit, is_maximization)

        # Print iteration details
        print(f"[$]Iteration: {iteration+1}")
        print("="*105)
        print("\tPopulation\t |\t  Objective Values\t|\t  Fitness Values\t|\tCounters")
        print("-"*105)
        for i in range(Np):
            print(f"{np.around(X[i], decimals=2)}| \t\t{round(f(*X[i]), 2)}\t\t|\t\t{round(fitness_function(f(*X[i])), 2)}\t\t|  \t  {int(trial[i])}")
        print("="*105)

    return best_solution, best_fit

##7. Main function


In [12]:
def main():

    # Run ABC algorithm for minimization
    best_solution_min, best_fit_min = abc_algorithm(objective_function, lb, ub, Np, limit, max_iter, is_maximization=False)
    print("As a Maximization Problem:")
    print("-"*27)
    print("Best Solution:", [f"{val:.2f}" for val in best_solution_min])
    print("Best Fitness:", round(best_fit_min, 2))
    print("\n\n")
    print("-"*105)
    print("~"*105)
    print("-"*105)
    print("\n\n")


    # Run ABC algorithm for maximization
    best_solution_max, best_fit_max = abc_algorithm(objective_function, lb, ub, Np, limit, max_iter, is_maximization=True)
    print("As a Minimization Problem:")
    print("-"*27)
    print("Best Solution:", [f"{val:.2f}" for val in best_solution_max])
    print("Best Fitness:", round(best_fit_max, 2))
    print("-"*105)
    print("~"*105)
    print("-"*105)

In [13]:
if __name__ == "__main__":
    main()

[$]Iteration: 1
	Population	 |	  Objective Values	|	  Fitness Values	|	Counters
---------------------------------------------------------------------------------------------------------
[-0.67  3.62  0.69 -5.94]| 		49.4		|		0.02		|  	  3
[-1.54 -3.71  5.85 -4.51]| 		70.73		|		0.01		|  	  0
[-7.    2.23  2.2  -6.96]| 		107.37		|		0.01		|  	  2
[-9.98  0.64 -5.28  1.5 ]| 		130.22		|		0.01		|  	  1
[-5.45  3.84  8.17  5.21]| 		138.31		|		0.01		|  	  0
[$]Iteration: 2
	Population	 |	  Objective Values	|	  Fitness Values	|	Counters
---------------------------------------------------------------------------------------------------------
[-0.67  1.71  0.69 -5.94]| 		39.19		|		0.02		|  	  2
[-0.67 -3.71  5.85 -4.51]| 		68.82		|		0.01		|  	  0
[-7.    2.23  2.2  -6.96]| 		107.37		|		0.01		|  	  3
[-8.54  0.64 -5.28  1.5 ]| 		103.54		|		0.01		|  	  0
[-5.45  3.84  3.88  5.21]| 		86.67		|		0.01		|  	  0
[$]Iteration: 3
	Population	 |	  Objective Values	|	  Fitness Values	|	Counters
--------------

##Conclusion:


The implementation demonstrates the effectiveness of the ABC algorithm in solving optimization problems. Through multiple iterations, the algorithm refines the population of solutions, gradually improving their fitness values. The algorithm efficiently explores the solution space by balancing exploration and exploitation, ensuring convergence towards the global optimum.

In the provided output, we observe the evolution of the population over successive iterations, with each iteration bringing the algorithm closer to the optimal solution. The final solution obtained showcases the capability of the ABC algorithm to find optimal or near-optimal solutions for both minimization and maximization problems.

With further fine-tuning and adaptation to specific problem domains, the ABC algorithm can be a powerful tool for solving a wide range of optimization problems in various fields, including engineering, economics, and machine learning